<a href="https://colab.research.google.com/github/MatthewYancey/GANime/blob/master/src/train_lama_default.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tain LaMa
This notebok runs the default training for LaMa

In [ ]:
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

In [ ]:
import os
import shutil
import glob

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
dataset_dir = '/content/lama/my_dataset/'
train_zip = '/content/gdrive/MyDrive/repos/GANime/data_out/pokemon/train.zip'
val_zip = '/content/gdrive/MyDrive/repos/GANime/data_out/pokemon/validate.zip'
test_zip = '/content/gdrive/MyDrive/repos/GANime/data_out/pokemon/test.zip'

In [ ]:
print('\n> Cloning the repo')
!git clone https://github.com/saic-mdal/lama.git

print('\n> Install dependencies')
!pip install -r lama/requirements.txt --quiet
!pip install wget --quiet

print('\n> Changing the dir to:')
% cd /content/lama

# downloads the models for evaluations
print('Downloading resnet model for evaluation')
!mkdir -p ade20k/ade20k-resnet50dilated-ppm_deepsup/
!wget -P ade20k/ade20k-resnet50dilated-ppm_deepsup/ http://sceneparsing.csail.mit.edu/model/pytorch/ade20k-resnet50dilated-ppm_deepsup/encoder_epoch_20.pth

print('>fixing opencv')
!pip uninstall opencv-python-headless -y --quiet
!pip install opencv-python-headless==4.1.2.30 --quiet

print('\n> Init mask-drawing code')
import base64, os
from IPython.display import HTML, Image
from google.colab.output import eval_js
from base64 import b64decode
import matplotlib.pyplot as plt
import numpy as np
from shutil import copyfile
import shutil
import cv2

In [ ]:
# copy the dataset set over
try:
    os.mkdir(dataset_dir)
except FileExistsError:
    pass

print('Copying and unpacking training data')
shutil.copy(train_zip, dataset_dir + 'train.zip')
shutil.unpack_archive(dataset_dir + 'train.zip', dataset_dir + '/train/')
print('Copying and unpacking validation data')
shutil.copy(val_zip, dataset_dir + 'val.zip')
shutil.unpack_archive(dataset_dir + 'val.zip', dataset_dir + '/val_original/')
print('Copying and unpacking testing data')
shutil.copy(test_zip, dataset_dir + 'test.zip')
shutil.unpack_archive(dataset_dir + 'test.zip', dataset_dir + '/visual_test_original/')

In [ ]:
# makes the config file for training
!touch my_dataset.yaml
!echo "data_root_dir: $(pwd)/my_dataset/" >> my_dataset.yaml
!echo "out_root_dir: $/content/gdrive/MyDrive/repos/GANime/data_out/lama_model/" >> my_dataset.yaml
!echo "tb_dir: $(pwd)/tb_logs/" >> my_dataset.yaml
!mv my_dataset.yaml ${PWD}/configs/training/location/

# change /content/lama/configs/training/trainer/any_gpu_large_ssim_ddp_final.yaml limit_train_batches: 2600 # was 25000
# shutil.copy('/content/gdrive/MyDrive/repos/GANime/src/any_gpu_large_ssim_ddp_final.yaml', '/content/lama/configs/training/trainer/any_gpu_large_ssim_ddp_final.yaml')

In [ ]:
# produces the masks
def prep_data(in_dir, out_dir, img_width, img_height, single_side):
    # creates a test set of validaiton
    if os.path.isdir(out_dir):
        shutil.rmtree(out_dir)
    os.mkdir(out_dir)

    img_paths = glob.glob(in_dir + '*')

    # makes a mask and saves it to correspond with the names of images
    inside = np.zeros((img_height, img_width - 2*single_side, 3), np.uint8)
    outsides = np.ones((img_height, single_side, 3), np.uint8) * 255
    mask = cv2.hconcat([outsides, inside, outsides])

    for img_path in img_paths:
        # converts image
        img = cv2.imread(img_path)
        new_img_path = out_dir + os.path.basename(img_path).replace('.jpg', '.png')
        cv2.imwrite(new_img_path, img)

        # saves mask
        mask_name = new_img_path.replace('.png', '') + '_mask.png'
        cv2.imwrite(mask_name, mask)

In [ ]:
prep_data(dataset_dir + 'val_original/', dataset_dir + 'val/', 512, 288, 64)
prep_data(dataset_dir + 'visual_test_original/', dataset_dir + 'visual_test/', 512, 288, 64)

In [ ]:
!export TORCH_HOME=$(pwd) && export USER='matt' && export PYTHONPATH=. && python3 bin/train.py -cn lama-fourier location=my_dataset data.batch_size=10
# !cat ${PWD}/configs/training/data/abl-04-256-mh-dist

In [ ]:
# shutil.make_archive('/content/gdrive/MyDrive/lama_model', 'zip', '/content/lama/experiments/matt_2022-02-12_20-17-09_train_lama-fourier_')